In [1]:
Datasets_url = "https://www.kaggle.com/datasets/muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten

Dataset URL: https://www.kaggle.com/datasets/muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten
License(s): CC0-1.0
100% 4.77G/4.77G [00:23<00:00, 191MB/s]
100% 4.77G/4.77G [00:23<00:00, 214MB/s]


In [4]:
!unzip -q fruit-and-vegetable-disease-healthy-vs-rotten.zip

In [5]:
# Import
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms, models
from PIL import Image
from tqdm.auto import tqdm
import time

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [7]:
# Define transformations (e.g., resize, normalization)
transform = transforms.Compose([
    transforms.Resize((255,255)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Gather file paths of all images in the folders and their corresponding labels
        for label, folder in enumerate(os.listdir(root_dir)):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                  if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                    self.samples.append((os.path.join(folder_path, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        try:
            image_path, label = self.samples[index]
            image = Image.open(image_path).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image, label

        except Exception as e:
            # If there's an error opening or processing the image, print the error and return None
            #print(f"Error loading image: {image_path} - {e}")
            return self.__getitem__(index + 1)

In [9]:
# Instantiate custom dataset
root_dir = '/content/Fruit And Vegetable Diseases Dataset'
custom_dataset = CustomImageDataset(root_dir, transform=transform)

In [10]:
len(custom_dataset)

29291

In [11]:
custom_dataset[0]

(tensor([[[-0.0039, -0.0588, -0.1137,  ..., -0.6314, -0.6392, -0.6471],
          [-0.0196, -0.0510, -0.0980,  ..., -0.6235, -0.6314, -0.6392],
          [-0.0431, -0.0667, -0.1059,  ..., -0.6157, -0.6235, -0.6314],
          ...,
          [ 0.4510,  0.4745,  0.4667,  ...,  0.3020,  0.3020,  0.1686],
          [ 0.4353,  0.4667,  0.4588,  ...,  0.2784,  0.2078,  0.1137],
          [ 0.4275,  0.4667,  0.4667,  ...,  0.2941,  0.1686,  0.1137]],
 
         [[ 0.0039, -0.0431, -0.0902,  ..., -0.6235, -0.6314, -0.6392],
          [-0.0118, -0.0275, -0.0667,  ..., -0.6157, -0.6235, -0.6314],
          [-0.0196, -0.0353, -0.0510,  ..., -0.6078, -0.6157, -0.6235],
          ...,
          [-0.1843, -0.1529, -0.1451,  ..., -0.1294, -0.0902, -0.2000],
          [-0.1686, -0.1294, -0.1137,  ..., -0.1137, -0.1529, -0.2235],
          [-0.1686, -0.1137, -0.0980,  ..., -0.0980, -0.1843, -0.2078]],
 
         [[-0.0588, -0.1137, -0.1608,  ..., -0.6627, -0.6706, -0.6784],
          [-0.0745, -0.0980,

In [12]:
# Example usage
sample_image, sample_label = custom_dataset[0]
print(sample_image.shape)  # Check the shape of the image tensor
print(sample_label)  # Check the label of the image

torch.Size([3, 255, 255])
0


In [13]:
# Define the size of the training set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size

In [14]:
# Split the dataset into training and testing sets
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

In [15]:
len(train_dataset), len(test_dataset)

(23432, 5859)

In [16]:
# Define batch size for DataLoader
batch_size = 16

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for testing set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.to(device)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    model.to(device)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
# Create a convolutional neural network
class CnnModel(nn.Module):
    """
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2),
            nn.Dropout(0.2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=198450,
                      out_features=hidden_units*10*10),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units*10*10),
            nn.Linear(in_features=hidden_units*10*10,
                      out_features=hidden_units*5*5),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units*5*5),
            nn.Linear(in_features=hidden_units*5*5,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units),
            nn.Linear(in_features=hidden_units,
                      out_features=output_shape),
            nn.Softmax(dim=1)  # Add softmax layer
        )

    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

torch.manual_seed(23)
model = CnnModel(input_shape=3,
    hidden_units=50,
    output_shape=28)
model

CnnModel(
  (block_1): Sequential(
    (0): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.2, inplace=False)
  )
  (block_2): Sequential(
    (0): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.2, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=198450, out_features=5000, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=5000, out_features=1250, bias=True)
    (5): ReLU()


In [23]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                             lr=0.05)

In [24]:
from tqdm.auto import tqdm
import time
start_time = time.time()


# Train and test model
epochs = 5
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_loader, model, loss_fn, optimizer,device)
    test(test_loader, model, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 3.337879  [   16/23432]
loss: 3.189659  [ 1616/23432]
loss: 3.181325  [ 3216/23432]
loss: 3.069927  [ 4816/23432]
loss: 3.264855  [ 6416/23432]
loss: 3.140426  [ 8016/23432]
loss: 3.018093  [ 9616/23432]
loss: 3.133760  [11216/23432]
loss: 3.010782  [12816/23432]
loss: 3.041294  [14416/23432]
loss: 3.186072  [16016/23432]
loss: 2.718027  [17616/23432]
loss: 2.995129  [19216/23432]
loss: 3.025258  [20816/23432]
loss: 2.890819  [22416/23432]
Test Error: 
 Accuracy: 42.0%, Avg loss: 2.979065 

Epoch: 1
---------
loss: 3.088448  [   16/23432]
loss: 3.073464  [ 1616/23432]
loss: 3.030395  [ 3216/23432]
loss: 2.719288  [ 4816/23432]
loss: 3.012253  [ 6416/23432]
loss: 2.962013  [ 8016/23432]
loss: 2.980316  [ 9616/23432]
loss: 2.638641  [11216/23432]
loss: 3.046119  [12816/23432]
loss: 2.861777  [14416/23432]
loss: 2.943599  [16016/23432]
loss: 2.904335  [17616/23432]
loss: 2.766748  [19216/23432]
loss: 3.003991  [20816/23432]
loss: 2.752727  [22416/23432]
Test Error